<a href="https://colab.research.google.com/github/UrtziAzkarate/Applied-Data-Science-with-Python-Course-1/blob/main/skin_cancer_dataset_malign_vs_benign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.utils.data import random_split

In [ ]:
# Si tenemos la GPU disponible, utilizamos la GPU para que las imágenes de cada batch se procesen paralelamente de forma más eficiente. De lo contrario utilizamos la CPU disponible.

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device : {device}')

Using device : cpu


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fanconic/skin-cancer-malignant-vs-benign")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'skin-cancer-malignant-vs-benign' dataset.
Path to dataset files: /kaggle/input/skin-cancer-malignant-vs-benign


**IMPORTAMOS DRIVE PARA PODER CONECTARNOS AL DATASET DESCARGADO DE KAGGLE EN DRIVE**

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


  **PUNTOS 1 y 2**

*   Leemos y cargámos las imágenes en Tensores de PyTorch ya partidos en TRAIN y TEST

In [ ]:
path_train = '/content/drive/MyDrive/MSc Inteligencia Artificial/Modulo 6 - Visión Artificial/Actividades/Actividades Entregables/Skin Cancer: Malign vs Benign/archive/data/train'
path_test =  '/content/drive/MyDrive/MSc Inteligencia Artificial/Modulo 6 - Visión Artificial/Actividades/Actividades Entregables/Skin Cancer: Malign vs Benign/archive/data/test'

train_dataset = datasets.ImageFolder(root=path_train)
test_dataset = datasets.ImageFolder(root=path_test)

**PUNTO 2.**

*   Divide el conjunto de datos en dos: uno para entrenamiento y otro para test.
Esto ya no nos hace falta hacer ya que en origen el dataset viene particionado.



In [ ]:
# train_ratio = 0.8
# train_size = int(train_ratio * len(cancer_data))
# test_size = len(cancer_data) - train_size
# train_dataset, test_dataset = random_split(cancer_data, [train_size, test_size])

# print(train_size)
# print(test_size)


**PUNTO 3.**

*  Aplicamos técnicas de Data Augmentation como el desplazamiento horizontal y la rotación de ±20°



In [ ]:
## IMPORTANTE ##
# PRIMERO TRANSFORMAMOS EL TRAIN APLICADO DATA AUGMENTATION, DESPUÉS AL TEST NO LE APLICAREMOS EL DATA AUGMENTATION, TAN SOLO LA NORMALIZACIÓN. Ya que el data augmentation se utiliza para entrenar. ##


# Comprimimos el tamaño de las imagenes a 224x224 que es una medida estandar que permite:
# 1. Entrenar modelos sin perder detalles importantes
# 2. Entrenar modelos de forma rápida y óptima

# Aplicamos técnicas de aumentación de movimiento horizontal aleatorio de la imágen así como la rotación aleatoria de la imagen en ±20°

# Aplicamos una normalización de las variables para que los valores vayan de -1 a 1. Esto nos ayudará a mejorar el aprendizaje del modelo, ya que
# Cuando los valores se centran alrededor del 0, los modelos son muchos más eficientes a la hora de aplicar gradientes o funciones de activación como el ReLU, donde los valores negativos (-) se desactivan.


transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5), # reflejo horizontal aleatorio
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.05)),  # Desplaza la imagen horizontalmente hasta un 10% y verticalmente hasta un 5% de la anchura de la imágen.
    transforms.RandomCrop(size=(224, 224), padding=10), # Añadimos un padding de 10 para después recortar la imagen e intentar entrenar el modelo con las zonas más importantes de cada imágen
    transforms.RandomRotation(degrees=20), # rotación aleatoria ±20°
    transforms.Normalize(mean=[0.5, 0.5, 0.5], # Coge cada valor X (que va de 0 a 1 después de haber aplicado el ToTensor) y hace la siguiente ecuación para convertir valores entre -1 y 1: (x-0.5)/0.5
                         std=[0.5, 0.5, 0.5]),

])


transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], # Coge cada valor X (que va de 0 a 1 después de haber aplicado el ToTensor) y hace la siguiente ecuación para convertir valores entre -1 y 1: (x-0.5)/0.5
                         std=[0.5, 0.5, 0.5]),

])

In [ ]:
# Aplicamos las transformaciones diseñadas en el paso anterior

# train_dataset.dataset.transform = transform_train --> Esto lo haríamos en caso de que el train_dataset aún NO fuese instancia de ImageFolder (e.g., cuando se utiliza random split para dividir el dataset)
# test_dataset.dataset.transform = transform_test

train_dataset = datasets.ImageFolder(root=path_train, transform=transform_train)
test_dataset = datasets.ImageFolder(root=path_test, transform=transform_test)

# Creamos un loader para configurar los batches, un batch_size de 44 significa que utilizamos lotes de 44 imágenes, donde todas las imágenes de cada lote se ejecutan a la vez.
# Esto nos permite que el gradiente se calcule de forma menos ruidosa, es decir, a la hora de recalcular los pesos para minimizar los errores, tendrá en cuenta el promedio del error (cuadrático?) de todas las imágenes, en vez de una sola imágen.
# Si tenemos 132 imágenes y el tamaño de los batches es de 44, tendríamos un total de 3 batches para procesar todas las imágenes. Cuanto mayor sea el batch_size, mayor tardará en ejecutarse cada batch.

train_loader = DataLoader(train_dataset, batch_size=44, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=44, shuffle=False)

# Leemos los batches de imágenes como tensores para después poder entrenar el modelo
images, labels = next(iter(train_loader))

print("Shape del set de Train:", len(train_dataset))
print("Shape del set de Test:", len(test_dataset))
print("Shape del set del Dataset:", len(train_dataset) + len(test_dataset))
print("Shape del tensor de imágenes:", images.shape)
print("Shape del tensor de labels:", labels.shape)

Shape del set de Train: 2637
Shape del set de Test: 660
Shape del set del Dataset: 3297
Shape del tensor de imágenes: torch.Size([44, 3, 224, 224])
Shape del tensor de labels: torch.Size([44])


**PUNTO 4.**


*   Instanciamos una Deep Neural Network de ResNet (ImageNet), descargando los pesos preestablecidos por defecto del propio modelo.



In [ ]:
resnet18_model = models.resnet18(weights="DEFAULT")  # pesos preentrenados de ImageNet
num_features = resnet18_model.fc.in_features  # número de features de la última capa

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 112MB/s]


**PUNTO 5.**


*   Eliminamos las capas Lineales de la etapa clasificadora de la red neuronal y
sobrescrimos con unas nuevas capas lineales con los mismos parámetros de
configuración ajustadas a nuestra problemática



In [ ]:
# Reemplazamos la cabeza clasificadora.
# La ResNet18 produce 512 features y 1000 clases en la última capa convolucional antes de llegar a las capas densas.
# Nosotros tenemos que ajustar la salida a 512 features (lo dejamos como está) pero a 2 clases (Benigno vs Maligno)

resnet18_model.fc = nn.Linear(num_features, 2)  # 2 clases: benigno vs maligno

resnet18_model = resnet18_model.to(device) # Se entrena con la GPU o CPU, según lo que tengamos disponible

**PUNTO 6.**


*   Realizamos el entrenamiento de la red neuronal configurada. (Con la nueva cabeza clasificadora) actualizando solo los pesos de las capas nuevas.



In [ ]:
# Solo entrenaremos los parámetros de la nueva cabeza clasificadora (el que decide si es Benigno o Maligno)
for param in resnet18_model.parameters():
    param.requires_grad = False # No queremos que el modelo (capas convolucionales y capas densas) reajuste sus pesos, ya que es un modelo preentrenado
for param in resnet18_model.fc.parameters():
    param.requires_grad = True # Solo queremos reajustar la parte final, la cabeza clasificadora

criterion = nn.CrossEntropyLoss() # Aplicamos CrossEntropyLoss para ajustar los pesos y reducir el loss
optimizer = optim.Adam(resnet18_model.fc.parameters(), lr=1e-3)

num_epochs = 5  # Vamos a utilizar 5 epochs (iteraciones completas) para entrenar al modelo. 5 Epochs y (2637/44) batches. En cada batch y cada epoch los pesos se reajustan.

for epoch in range(num_epochs):
    resnet18_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad() # Reseteamos los gradientes en cada batch para que no se vayan acumulando. Esto evita que los gradientes se vayan inflando de forma masiva.
        outputs = resnet18_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.4f}")

Epoch 1/5 - Loss: 0.5193 - Accuracy: 0.7342
Epoch 2/5 - Loss: 0.4410 - Accuracy: 0.7835
Epoch 3/5 - Loss: 0.3874 - Accuracy: 0.8187
Epoch 4/5 - Loss: 0.3878 - Accuracy: 0.8225
Epoch 5/5 - Loss: 0.3690 - Accuracy: 0.8335


**PUNTO 7.**

*   Determinamos las métricas de función de pérdida (loss) y precisión para los conjuntos de datos de entrenamiento y test.



In [ ]:
resnet18_model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet18_model(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= total
test_acc = correct / total
print(f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_acc:.4f}")

Test Loss: 0.4360 - Test Accuracy: 0.8242
